# Koosta klassifikatsioonimudel: Maitsvad Aasia ja India köögid


## Köögi klassifikaatorid 2

Selles teises klassifikatsiooniõppetunnis uurime `rohkem viise`, kuidas kategoriseerida andmeid. Samuti õpime, millised on tagajärjed ühe klassifikaatori valimisel teise asemel.

### [**Eelloengu viktoriin**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Eeltingimus**

Eeldame, et olete läbinud eelnevad õppetunnid, kuna jätkame mõningate varem õpitud kontseptsioonidega.

Selleks õppetunniks vajame järgmisi pakette:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) on [R-pakettide kogum](https://www.tidyverse.org/packages), mis on loodud selleks, et muuta andmeteadus kiiremaks, lihtsamaks ja lõbusamaks!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) raamistik on [pakettide kogum](https://www.tidymodels.org/packages/), mis on mõeldud modelleerimiseks ja masinõppeks.

-   `themis`: [themis pakett](https://themis.tidymodels.org/) pakub lisaretsepte tasakaalustamata andmetega tegelemiseks.

Saate need paigaldada järgmiselt:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Alternatiivselt kontrollib allolev skript, kas teil on selle mooduli läbimiseks vajalikud paketid olemas, ja paigaldab need vajadusel.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

Nüüd alustame hoogsalt!

## **1. Klassifikatsioonikaart**

Meie [eelmises tunnis](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) püüdsime vastata küsimusele: kuidas valida mitme mudeli vahel? Suures osas sõltub see andmete omadustest ja probleemist, mida soovime lahendada (näiteks klassifikatsioon või regressioon?).

Varem õppisime erinevate võimaluste kohta, mis on olemas andmete klassifitseerimiseks, kasutades Microsofti spikrit. Pythoni masinõppe raamistik Scikit-learn pakub sarnast, kuid detailsemat spikrit, mis aitab veelgi täpsemalt valida sobivaid hindajaid (teine termin klassifikaatorite kohta):

<p >
   <img src="../../../../../../translated_images/map.e963a6a51349425ab107b38f6c7307eb4c0d0c7ccdd2e81a5e1919292bab9ac7.et.png"
   width="700"/>
   <figcaption></figcaption>


> Näpunäide: [vaata seda kaarti veebis](https://scikit-learn.org/stable/tutorial/machine_learning_map/) ja klõpsa teekonnal, et lugeda dokumentatsiooni.
>
> [Tidymodels'i viitesait](https://www.tidymodels.org/find/parsnip/#models) pakub samuti suurepärast dokumentatsiooni erinevate mudelitüüpide kohta.

### **Plaani ülevaade** 🗺️

See kaart on väga kasulik, kui sul on oma andmetest selge arusaam, kuna saad selle teekondadel "kõndida" ja teha otsuseid:

-   Meil on \>50 näidist

-   Me tahame ennustada kategooriat

-   Meil on märgistatud andmed

-   Meil on vähem kui 100K näidist

-   ✨ Saame valida Linear SVC

-   Kui see ei tööta, kuna meil on numbrilised andmed

    -   Võime proovida ✨ KNeighbors Classifier'it

        -   Kui see ei tööta, proovime ✨ SVC ja ✨ Ensemble Classifiers'it

See on väga kasulik teekond, mida järgida. Nüüd, asume asja kallale, kasutades [tidymodels](https://www.tidymodels.org/) modelleerimisraamistikku: järjepidevat ja paindlikku R-pakettide kogumit, mis on loodud hea statistilise praktika edendamiseks 😊.

## 2. Andmete jagamine ja tasakaalustamata andmekogumiga tegelemine.

Eelmistest tundidest õppisime, et meie köökide vahel oli ühiseid koostisosi. Samuti oli köökide arv jaotunud üsna ebaühtlaselt.

Sellega tegeleme järgmiselt:

-   Kõige tavalisemate koostisosade eemaldamine, mis tekitavad segadust erinevate köökide vahel, kasutades `dplyr::select()`.

-   Kasutame `recipe`'i, mis eeltöötleb andmeid, et need modelleerimiseks valmis seada, rakendades `üle-sampling` algoritmi.

Oleme eelnevalt neid samme juba käsitlenud, seega peaks see olema lihtne 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Kuidas toime tulla tasakaalustamata andmetega

Tasakaalustamata andmed mõjutavad sageli mudeli jõudlust negatiivselt. Paljud mudelid töötavad kõige paremini, kui vaatluste arv on võrdne, ja seetõttu kipuvad nad tasakaalustamata andmetega hätta jääma.

Tasakaalustamata andmekogumitega toimetulemiseks on peamiselt kaks viisi:

-   vähemuses oleva klassi vaatlustele lisamine: `Üle-sampling`, näiteks SMOTE algoritmi kasutamine, mis sünteetiliselt genereerib vähemuses oleva klassi uusi näiteid, kasutades nende juhtumite lähimaid naabreid.

-   enamuses oleva klassi vaatlustest eemaldamine: `Alam-sampling`

Eelmises tunnis näitasime, kuidas tasakaalustamata andmekogumitega toime tulla, kasutades `retsepti`. Retsepti võib mõelda kui plaani, mis kirjeldab, milliseid samme tuleks andmekogumile rakendada, et see oleks valmis andmeanalüüsiks. Meie puhul soovime, et meie `treeningkogumis` oleks köökide arv ühtlaselt jaotatud. Asume asja kallale.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Nüüd oleme valmis mudeleid treenima 👩‍💻👨‍💻!

## 3. Multinomiaalse regressiooni mudelitest kaugemale

Eelmises tunnis vaatasime multinomiaalse regressiooni mudeleid. Uurime nüüd mõningaid paindlikumaid klassifitseerimismudeleid.

### Toetavate vektorite masinad

Klassifitseerimise kontekstis on `Toetavate vektorite masinad` masinõppe meetod, mis püüab leida *hüpertasandi*, mis "kõige paremini" eraldab klassid. Vaatame lihtsat näidet:

<p >
   <img src="../../../../../../translated_images/svm.621ae7b516d678e08ed23af77ff1750b5fe392976917f0606861567b779e8862.et.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ ei eralda klasse. H2~ eraldab, kuid ainult väikese vahega. H3~ eraldab maksimaalse vahega.

#### Lineaarne toetusvektori klassifikaator

Toetusvektori klasterdamine (SVC) kuulub masinõppe tehnikate toetusvektori masinate perekonda. SVC-s valitakse hüpertasand, mis eraldab `enamiku` treeningandmetest korrektselt, kuid `võib valesti klassifitseerida` mõned andmepunktid. Lubades mõnel punktil olla valel poolel, muutub SVM vastupidavamaks kõrvalekallete suhtes ja seega paremaks uute andmete üldistamisel. Parameetrit, mis reguleerib seda rikkumist, nimetatakse `kuluks`, mille vaikeväärtus on 1 (vt `help("svm_poly")`).

Loome lineaarse SVC, määrates polünoomse SVM-i mudelis `degree = 1`.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Nüüd, kui oleme salvestanud eeltöötluse sammud ja mudeli spetsifikatsiooni *töövoogu*, saame jätkata lineaarse SVC treenimist ja samal ajal tulemusi hinnata. Jõudlusmõõdikute jaoks loome mõõdikute komplekti, mis hindab: `täpsust`, `tundlikkust`, `positiivset prognoositud väärtust` ja `F-mõõdet`.

> `augment()` lisab antud andmetele veeru(d) ennustuste jaoks.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Toetuste vektormasin

Toetuste vektormasin (SVM) on toetuste vektorklassifikaatori laiendus, mis võimaldab klasside vahel mittelineaarset piiri. Sisuliselt kasutavad SVM-id *tuuma trikki*, et suurendada tunnuste ruumi ja kohanduda klasside vaheliste mittelineaarsete seostega. Üks populaarne ja äärmiselt paindlik tuumafunktsioon, mida SVM-id kasutavad, on *radiaalne baasfunktsioon.* Vaatame, kuidas see meie andmetel toimib.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Palju parem 🤩!

> ✅ Palun vaata:
>
> -   [*Toetavate vektorite masinad*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Toetavate vektorite masinad*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> lisalugemiseks.

### Lähima naabri klassifikaatorid

*K*-lähima naabri (KNN) algoritm ennustab iga vaatluse põhjal selle *sarnasuse* teiste vaatluste suhtes.

Sobitame ühe meie andmetele.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Tundub, et see mudel ei tööta eriti hästi. Tõenäoliselt mudeli argumentide muutmine (vaata `help("nearest_neighbor")`) parandab mudeli jõudlust. Kindlasti proovi seda.

> ✅ Vaata:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> et rohkem teada saada *K*-lähimate naabrite klassifikaatorite kohta.

### Ansambli klassifikaatorid

Ansambli algoritmid töötavad mitme baasmudeli kombineerimise teel, et luua optimaalne mudel, kas:

`bagging`: rakendades *keskmistamisfunktsiooni* baasmudelite kogumile

`boosting`: luues järjestikuseid mudeleid, mis üksteist täiendavad, et parandada ennustusvõimet.

Alustame Random Foresti mudeli proovimisega, mis loob suure kogumi otsustuspuud ja rakendab keskmistamisfunktsiooni, et saada parem üldine mudel.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Tubli töö 👏!

Proovime nüüd ka Boosted Tree mudelit.

Boosted Tree on ansamblimeetod, mis loob järjestikuste otsustuspuude seeria, kus iga puu sõltub eelnevate puude tulemustest, püüdes järk-järgult vähendada viga. See keskendub valesti klassifitseeritud elementide kaaludele ja kohandab järgmise klassifikaatori sobivust, et viga parandada.

Selle mudeli sobitamiseks on erinevaid viise (vaata `help("boost_tree")`). Selles näites sobitame Boosted Tree mudeli `xgboost` mootoriga.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Palun vaata:
>
> -   [Masinõpe sotsiaalteadlastele](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Praktiline masinõpe R-iga](https://bradleyboehmke.github.io/HOML/)
>
> -   [Statistilise õppimise sissejuhatus rakendustega R-is](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Uurib AdaBoost mudelit, mis on hea alternatiiv xgboostile.
>
> et rohkem teada saada ansambelklassifikaatorite kohta.

## 4. Lisa - mitme mudeli võrdlemine

Oleme selles laboris sobitanud üsna palju mudeleid 🙌. Erinevate eeltöötluskomplektide ja/või mudelispetsifikatsioonidega paljude töövoogude loomine ning seejärel jõudlusmõõdikute ükshaaval arvutamine võib muutuda tüütuks või koormavaks.

Vaatame, kas saame seda lahendada, luues funktsiooni, mis sobitab koolitusandmetele töövoogude loendi ja tagastab testandmete põhjal jõudlusmõõdikud. Kasutame selleks [purrr](https://purrr.tidyverse.org/) paketist funktsioone `map()` ja `map_dfr()`, et rakendada funktsioone iga loendi elemendi suhtes.

> [`map()`](https://purrr.tidyverse.org/reference/map.html) funktsioonid võimaldavad asendada paljud for-tsüklid koodiga, mis on nii lühem kui ka lihtsam lugeda. Parim koht [`map()`](https://purrr.tidyverse.org/reference/map.html) funktsioonide kohta õppimiseks on [iteratsiooni peatükk](http://r4ds.had.co.nz/iteration.html) raamatus "R for Data Science".


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

Kutsume oma funktsiooni ja võrdleme mudelite täpsust.


In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) pakett võimaldab kasutajatel luua ja hõlpsasti sobitada suurt hulka mudeleid, kuid on peamiselt mõeldud töötamiseks resampleerimistehnikatega, nagu `ristvalideerimine`, meetod, mida me pole veel käsitlenud.

## **🚀Väljakutse**

Igal neist tehnikatest on palju parameetreid, mida saab kohandada, näiteks `cost` SVM-ides, `neighbors` KNN-is, `mtry` (juhuslikult valitud ennustajad) Random Forestis.

Uuri iga mudeli vaikimisi parameetreid ja mõtle, mida nende parameetrite kohandamine võiks tähendada mudeli kvaliteedi jaoks.

Et saada rohkem teavet konkreetse mudeli ja selle parameetrite kohta, kasuta: `help("model")` näiteks `help("rand_forest")`.

> Praktikas *hinnatakse* tavaliselt *parimaid väärtusi* treenides palju mudeleid `simuleeritud andmekogumil` ja mõõtes, kui hästi kõik need mudelid toimivad. Seda protsessi nimetatakse **häälestamiseks**.

### [**Loengu järgne viktoriin**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Ülevaade ja iseseisev õppimine**

Nendes tundides on palju erialatermineid, seega võta hetk, et vaadata üle [see nimekiri](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) kasulikest mõistetest!

#### TÄNUD:

[`Allison Horst`](https://twitter.com/allison_horst/) suurepäraste illustratsioonide loomise eest, mis muudavad R-i kasutamise sõbralikumaks ja kaasahaaravamaks. Leia rohkem illustratsioone tema [galeriist](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) ja [Jen Looper](https://www.twitter.com/jenlooper) selle mooduli algse Python-versiooni loomise eest ♥️

Head õppimist,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../../../../../translated_images/r_learners_sm.f9199f76f1e2e49304b19155ebcfb8bad375aface4625be7e95404486a48d332.et.jpeg"
   width="569"/>
   <figcaption>Illustratsioon: @allison_horst</figcaption>



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
